In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from urllib.parse import urlencode
from urllib.request import urlopen
from datetime import datetime
import time
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
from statsmodels.tsa.api import SVAR
import statsmodels.api as sma
import warnings
warnings.filterwarnings('ignore')

1. нефть цепной
2. курс цепной
3. инфляция
4. ставка miacr
5. imoex цепной

- Скачать акции imoex
- проверить, что данные стационарны и нормированы
- VAR -> SVAR -> шоки
- $$Y_t = \alpha*Y_{t-1} + \beta*\epsilon_t + other \epsilon$$
- IRF

In [ ]:
!wget -O data https://docs.google.com/spreadsheets/d/1b7Px6ulbjW4-x_tJyoKPokYO-4zNpf-e2GLEXtmk4p0/export?output=xlsx

--2024-01-21 08:49:48--  https://docs.google.com/spreadsheets/d/1b7Px6ulbjW4-x_tJyoKPokYO-4zNpf-e2GLEXtmk4p0/export?output=xlsx
Resolving docs.google.com (docs.google.com)... 74.125.31.138, 74.125.31.101, 74.125.31.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.31.138|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-08-a8-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/vqsdn32tgsvt1jp4u6b436kkfo/1705826985000/101805398647211224213/*/1b7Px6ulbjW4-x_tJyoKPokYO-4zNpf-e2GLEXtmk4p0?output=xlsx [following]
--2024-01-21 08:49:48--  https://doc-08-a8-sheets.googleusercontent.com/export/54bogvaave6cua4cdnls17ksc4/vqsdn32tgsvt1jp4u6b436kkfo/1705826985000/101805398647211224213/*/1b7Px6ulbjW4-x_tJyoKPokYO-4zNpf-e2GLEXtmk4p0?output=xlsx
Resolving doc-08-a8-sheets.googleusercontent.com (doc-08-a8-sheets.googleusercontent.com)... 172.253.123.132, 2607:f8b0:400c:c16::84
Connecting to doc-08-a8-sheets.googleuserconte

In [ ]:
df = pd.read_excel('data', sheet_name="Итог_Арина", engine ="openpyxl")
df.tail(2)

,date,imoex,imoex_ind,sa_m2,sa_cpi_base,pui,nx,usd,eur,brent,ort,t10y3m,t10y2y,miacr_1d,reer,gold,sa_ort,sa_nx,inflation
248,2023-09-01,3133.26,10.180194,90864.4,NaN,0.0,NaN,96.62,103.3661,94.0,NaN,-0.45,-0.46,12.28,0.2,1915.95,NaN,NaN,NaN
249,NaT,3200.97,10.400188,NaN,NaN,NaN,NaN,99.52,104.9709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Загрузка акций

Можно скачать уже готовый файл https://github.com/arinaaandreeva/Finam_data/blob/main/Котировки_10_12_23.csv

In [ ]:
# Здесь я из табличек Мосбиржи получаю тикеры акций, которые были последние 5 лет в индексе
# Можно каждый раз их не грузить, просто внизу записала их в список
# таблицу можно найти тут: https://github.com/arinaaandreeva/Finam_data/blob/main/stock-index-base-moex-rts-18122012-nowadays.xlsx

#tick = pd.read_excel('/content/stock-index-base-moex-rts-18122012-nowadays.xlsx', sheet_name = 'Лист1')
#unique_values = tick.stack().unique()
#print(', '.join(map(lambda x: f'\'{x}\'', unique_values)))

In [ ]:
# all tickers for stocks, that were in imoex in last 5 years
tickers_name = ['SBER', 'GAZP', 'LKOH', 'SBERP', 'GMKN', 'NVTK', 'YNDX', 'TATN', 'TATNP', 'SNGS', 'POLY', 'ROSN', 'TCSG',
                'PLZL', 'SNGSP', 'MGNT', 'MTSS', 'ALRS', 'PHOR', 'PIKK', 'RUAL', 'VTBR', 'FIVE', 'NLMK', 'CHMF', 'IRAO',
                'MAIL', 'MAGN', 'OZON', 'RTKM', 'TRNFP', 'FIXP', 'DSKY', 'HYDR', 'AGRO', 'AFLT', 'POGR', 'VKCO', 'AFKS',
                'ENPG', 'MFON', 'MFON*', 'CBOM', 'HHRU', 'FEES', 'FLOT', 'RNFT', 'LNTA', 'LSRG', 'MTLR', 'UPRO', 'SFIN',
                'GLTR', 'TRMK', 'MVID', 'QIWI', 'SGZH', 'SELG', 'MTLRP', 'NMTP', 'RSTI', 'POSI', 'SMLT', 'UWGN', 'MSNG']

Задаем параментры экспорта акций

(можно сократить, возможно, но я не пыталась)

In [ ]:
period=10 # задаём период. Выбор из: 'tick': 1, 'min': 2, '5min': 3, '10min': 4, '15min': 5, '30min': 6, 'hour': 7, 'daily': 8, 'week': 9, 'month': 10
start = "01.01.2003" #с какой даты начинать тянуть котировки
end = "01.12.2023" #финальная дата, по которую тянуть котировки
########
periods={'tick': 1, 'min': 2, '5min': 3, '10min': 4, '15min': 5, '30min': 6, 'hour': 7, 'daily': 8, 'week': 9, 'month': 10}
#print ("ticker="+ticker+"; period="+str(period)+"; start="+start+"; end="+end)
#каждой акции Финам присвоил цифровой код:
tickers={'TCSG' : 913710,'ABRD':82460,'AESL':181867,'AFKS':19715,'AFLT':29,'AGRO':399716,'AKRN':17564,'ALBK':82616,'ALNU':81882,'ALRS':81820,'AMEZ':20702,'APTK':13855,'AQUA':35238,'ARMD':19676,'ARSA':19915,'ASSB':16452,'AVAN':82843,'AVAZ':39,'AVAZP':40,'BANE':81757,'BANEP':81758,'BGDE':175840,'BISV':35242,'BISVP':35243,'BLNG':21078,'BRZL':81901,'BSPB':20066,'CBOM':420694,'CHEP':20999,'CHGZ':81933,'CHKZ':21000,'CHMF':16136,'CHMK':21001,'CHZN':19960,'CLSB':16712,'CLSBP':16713,'CNTL':21002,'CNTLP':81575,'DASB':16825,'DGBZ':17919,'DIOD':35363,'DIXY':18564,'DVEC':19724,'DZRD':74744,'DZRDP':74745,'ELTZ':81934,'ENRU':16440,'EPLN':451471,'ERCO':81935,'FEES':20509,'FESH':20708,'FORTP':82164,'GAZA':81997,'GAZAP':81998,'GAZC':81398,'GAZP':16842,'GAZS':81399,'GAZT':82115,'GCHE':20125,'GMKN':795,'GRAZ':16610,'GRNT':449114,'GTLC':152876,'GTPR':175842,'GTSS':436120,'HALS':17698,'HIMC':81939,'HIMCP':81940,'HYDR':20266,'IDJT':388276,'IDVP':409486,'IGST':81885,'IGST03':81886,'IGSTP':81887,'IRAO':20516,'IRGZ':9,'IRKT':15547,'ISKJ':17137,'JNOS':15722,'JNOSP':15723,'KAZT':81941,'KAZTP':81942,'KBSB':19916,'KBTK':35285,'KCHE':20030,'KCHEP':20498,'KGKC':83261,'KGKCP':152350,'KLSB':16329,'KMAZ':15544,'KMEZ':22525,'KMTZ':81903,'KOGK':20710,'KRKN':81891,'KRKNP':81892,'KRKO':81905,'KRKOP':81906,'KROT':510,'KROTP':511,'KRSB':20912,'KRSBP':20913,'KRSG':15518,'KSGR':75094,'KTSB':16284,'KTSBP':16285,'KUBE':522,'KUNF':81943,'KUZB':83165,'KZMS':17359,'KZOS':81856,'KZOSP':81857,'LIFE':74584,'LKOH':8,'LNTA':385792,'LNZL':21004,'LNZLP':22094,'LPSB':16276,'LSNG':31,'LSNGP':542,'LSRG':19736,'LVHK':152517,'MAGE':74562,'MAGEP':74563,'MAGN':16782,'MERF':20947,'MFGS':30,'MFGSP':51,'MFON':152516,'MGNT':17086,'MGNZ':20892,'MGTS':12984,'MGTSP':12983,'MGVM':81829,'MISB':16330,'MISBP':16331,'MNFD':80390,'MOBB':82890,'MOEX':152798,'MORI':81944,'MOTZ':21116,'MRKC':20235,'MRKK':20412,'MRKP':20107,'MRKS':20346,'MRKU':20402,'MRKV':20286,'MRKY':20681,'MRKZ':20309,'MRSB':16359,'MSNG':6,'MSRS':16917,'MSST':152676,'MSTT':74549,'MTLR':21018,'MTLRP':80745,'MTSS':15523,'MUGS':81945,'MUGSP':81946,'MVID':19737,'NAUK':81992,'NFAZ':81287,'NKHP':450432,'NKNC':20100,'NKNCP':20101,'NKSH':81947,'NLMK':17046,'NMTP':19629,'NNSB':16615,'NNSBP':16616,'NPOF':81858,'NSVZ':81929,'NVTK':17370,'ODVA':20737,'OFCB':80728,'OGKB':18684,'OMSH':22891,'OMZZP':15844,'OPIN':20711,'OSMP':21006,'OTCP':407627,'PAZA':81896,'PHOR':81114,'PHST':19717,'PIKK':18654,'PLSM':81241,'PLZL':17123,'PMSB':16908,'PMSBP':16909,'POLY':175924,'PRFN':83121,'PRIM':17850,'PRIN':22806,'PRMB':80818,'PRTK':35247,'PSBR':152320,'QIWI':181610,'RASP':17713,'RBCM':74779,'RDRB':181755,'RGSS':181934,'RKKE':20321,'RLMN':152677,'RLMNP':388313,'RNAV':66644,'RODNP':66693,'ROLO':181316,'ROSB':16866,'ROSN':17273,'ROST':20637,'RSTI':20971,'RSTIP':20972,'RTGZ':152397,'RTKM':7,'RTKMP':15,'RTSB':16783,'RTSBP':16784,'RUAL':414279,'RUALR':74718,'RUGR':66893,'RUSI':81786,'RUSP':20712,'RZSB':16455,'SAGO':445,'SAGOP':70,'SARE':11,'SAREP':24,'SBER':3,'SBERP':23,'SELG':81360,'SELGP':82610,'SELL':21166,'SIBG':436091,'SIBN':2,'SKYC':83122,'SNGS':4,'SNGSP':13,'STSB':20087,'STSBP':20088,'SVAV':16080,'SYNG':19651,'SZPR':22401,'TAER':80593,'TANL':81914,'TANLP':81915,'TASB':16265,'TASBP':16266,'TATN':825,'TATNP':826,'TGKA':18382,'TGKB':17597,'TGKBP':18189,'TGKD':18310,'TGKDP':18391,'TGKN':18176,'TGKO':81899,'TNSE':420644,'TORS':16797,'TORSP':16798,'TRCN':74561,'TRMK':18441,'TRNFP':1012,'TTLK':18371,'TUCH':74746,'TUZA':20716,'UCSS':175781,'UKUZ':20717,'UNAC':22843,'UNKL':82493,'UPRO':18584,'URFD':75124,'URKA':19623,'URKZ':82611,'USBN':81953,'UTAR':15522,'UTII':81040,'UTSY':419504,'UWGN':414560,'VDSB':16352,'VGSB':16456,'VGSBP':16457,'VJGZ':81954,'VJGZP':81955,'VLHZ':17257,'VRAO':20958,'VRAOP':20959,'VRSB':16546,'VRSBP':16547,'VSMO':15965,'VSYD':83251,'VSYDP':83252,'VTBR':19043,'VTGK':19632,'VTRS':82886,'VZRZ':17068,'VZRZP':17067,'WTCM':19095,'WTCMP':19096,'YAKG':81917,'YKEN':81766,'YKENP':81769,'YNDX':388383,'YRSB':16342,'YRSBP':16343,'ZHIV':181674,'ZILL':81918,'ZMZN':556,'ZMZNP':603,'ZVEZ':82001}
FINAM_URL = "http://export.finam.ru/"# сервер, на который стучимся
market = 0 #можно не задавать. Это рынок, на котором торгуется бумага. Для акций работает с любой цифрой. Другие рынки не проверял.
#Делаем преобразования дат:
start_date = datetime.strptime(start, "%d.%m.%Y").date()
start_date_rev=datetime.strptime(start, '%d.%m.%Y').strftime('%Y/%m/%d')
end_date = datetime.strptime(end, "%d.%m.%Y").date()
end_date_rev=datetime.strptime(end, '%d.%m.%Y').strftime('%Y/%m/%d')

In [ ]:
# не для всех тикеров есть коды (их позже можно получить через код страницы). Поэтому пока выбираем те, что есть
# скорее всего, те компании, для которых нет кода, появились относительно недавно на бирже и данных для них тоже будет немного (где-то с 2019г)

poteryashki = []
for ticker in tickers_name:
  if ticker not in tickers:
    poteryashki.append(ticker)

tickers_name = [x for x in tickers_name if x not in poteryashki]

In [ ]:
len(tickers_name)

48

Загружаем котировки акций в финама 💥

In [ ]:
df_ticker = df[['date', 'imoex']]
df_ticker['date'] = pd.to_datetime(df_ticker['date'], format='%d/%m/%y')

for ticker in tickers_name:
  #Все параметры упаковываем в единую структуру. Здесь есть дополнительные параметры, кроме тех, которые заданы в шапке. См. комментарии внизу:
  params = urlencode([
            ('market', market), #на каком рынке торгуется бумага
            ('em', tickers[ticker]), #вытягиваем цифровой символ, который соответствует бумаге.
            ('code', ticker), #тикер нашей акции
            ('apply',0), #не нашёл что это значит.
            ('df', start_date.day), #Начальная дата, номер дня (1-31)
            ('mf', start_date.month - 1), #Начальная дата, номер месяца (0-11)
            ('yf', start_date.year), #Начальная дата, год
            ('from', start_date), #Начальная дата полностью
            ('dt', end_date.day), #Конечная дата, номер дня
            ('mt', end_date.month - 1), #Конечная дата, номер месяца
            ('yt', end_date.year), #Конечная дата, год
            ('to', end_date), #Конечная дата
            ('p', period), #Таймфрейм
            ('f', ticker+"_" + start_date_rev + "_" + end_date_rev), #Имя сформированного файла
            ('e', ".csv"), #Расширение сформированного файла
            ('cn', ticker), #ещё раз тикер акции
            ('dtf', 4), #В каком формате брать даты. Выбор из 5 возможных. См. страницу https://www.finam.ru/profile/moex-akcii/sberbank/export/
            ('tmf', 3), #В каком формате брать время. Выбор из 4 возможных.
            ('MSOR', 0), #Время свечи (0 - open; 1 - close)
            ('mstime', "on"), #Московское время
            ('mstimever', 1), #Коррекция часового пояса
            ('sep', 1), #Разделитель полей	(1 - запятая, 2 - точка, 3 - точка с запятой, 4 - табуляция, 5 - пробел)
            ('sep2', 1), #Разделитель разрядов
            ('datf', 1), #Формат записи в файл. Выбор из 6 возможных.
            ('at', 1)]) #Нужны ли заголовки столбцов


  url = FINAM_URL + ticker+"_" + start_date_rev + "_" + end_date_rev + ".csv?" + params #урл составлен!

  txt=urlopen(url).readlines() #здесь лежит огромный массив данных, прилетевший с Финама.
  local_file = open('quotes.csv', "w") #задаём файл, в который запишем котировки.
  for line in txt: #записываем свечи строку за строкой.
    local_file.write(line.strip().decode( "ISO-8859-1" )+'\n')
  local_file.close()
  df_new = pd.read_csv('/content/quotes.csv')
  df_new = df_new[['<DATE>','<CLOSE>']]
  df_new.rename(columns = {'<DATE>' : 'date', '<CLOSE>' : f'{ticker}'}, inplace = True)
  df_new['date'] = pd.to_datetime(df_new['date'], format='%d/%m/%y')

  df_ticker = pd.merge(df_ticker, df_new, on = 'date', how = 'left')



In [ ]:
df_ticker.head(2)

,date,imoex,SBER,GAZP,LKOH,SBERP,GMKN,NVTK,YNDX,TATN,...,UPRO,TRMK,MVID,QIWI,SELG,MTLRP,NMTP,RSTI,UWGN,MSNG
0,2003-01-01,307.78,6040.0,NaN,450.41,70.36,773.0,NaN,NaN,24.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.113
1,2003-02-01,341.52,6668.0,NaN,472.50,86.85,789.9,NaN,NaN,26.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.341


In [ ]:
# сколько пропусков, если 250 строк всего
# тут как раз видно, что код TCSG я добавляла сама руками и его акции только с 2019г.
#df_ticker.isna().sum()

# SVAR

In [ ]:
#df['brent'] = df['brent'] / df['brent'].shift(1)
#df['usd'] = df['usd'] / df['usd'].shift(1)
#df['imoex'] = df['imoex'] / df['imoex'].shift(1)

In [ ]:
df['brent'] = np.log(df['brent'] / df['brent'].shift(1))
#df['usd'] = np.log(df['usd'] / df['usd'].shift(1))
df['reer'] = np.log(df['reer']/100 + 1) # в таблице % и прирост
df['inflation'] = np.log(df['inflation'])
df['miacr_1d'] = df['miacr_1d'] - df['miacr_1d'].shift(1)
df['imoex'] = np.log(df['imoex'] / df['imoex'].shift(1))

In [ ]:
adfuller(df['miacr_1d'].dropna()) # все ряды стационарны, но не нормированы

(-16.050344248054532,
 5.783787492551557e-29,
 0,
 247,
 {'1%': -3.457105309726321,
  '5%': -2.873313676101283,
  '10%': -2.5730443824681606},
 793.7264351794013)

In [ ]:
data = df[['brent', 'reer', 'inflation', 'miacr_1d', 'imoex']].dropna()
model = VAR(data)
results_var = model.fit(1)

#results_var.summary()

In [ ]:
residuals = results_var.resid.to_numpy()
residuals

array([[-0.24326295,  0.00707555, -0.12053282, -0.06392275, -0.0874335 ],
       [-0.15798657,  0.01051985,  0.31075793, -0.93828169,  0.12481244],
       [ 0.09316709, -0.03272969,  0.23144964, -0.04283286,  0.09192257],
       ...,
       [-0.02083028, -0.03708798,  0.14452561,  0.38705723,  0.02108114],
       [ 0.07030376, -0.07160212,  0.6096985 ,  0.39154202,  0.07676053],
       [ 0.05398285, -0.03780581, -0.29093576,  2.38447571,  0.01207912]])

In [ ]:
#A = np.asarray([[1, 0, 0, 0, 0],['E', 1, 0, 0, 0],['E', 'E', 1, 0, 0],['E', 'E', 'E', 1, 0] ,['E', 'E', 'E','E', 1]])
#b4 = np.asarray([['E', 0, 0, 0], ['E', 'E', 0, 0], ['E', 'E', 'E', 0], ['E', 'E', 'E', 'E']])
b5 = np.asarray([['E', 0, 0, 0, 0], ['E', 'E', 0, 0, 0], ['E', 'E', 'E', 0, 0], ['E', 'E', 'E', 'E', 0], ['E', 'E', 'E', 'E', 'E']])

In [ ]:
svar_result = SVAR(data, svar_type='B', B=b5).fit(maxlags=1)


In [ ]:
matrix_p = svar_result.B
matrix_p

array([[ 9.85780546e-02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 4.65786821e-03, -3.64449922e-02,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-4.06577103e-02,  1.46939329e-01,  4.29594404e-01,
         0.00000000e+00,  0.00000000e+00],
       [-3.74775204e-02,  4.51478860e-01,  1.84202150e-01,
         1.32691443e+00,  0.00000000e+00],
       [ 2.88761209e-02,  7.93529235e-03, -2.96995889e-04,
        -5.08136665e-03,  6.77478166e-02]])

In [ ]:
shocks = np.dot(residuals, np.linalg.inv(matrix_p))  # транспонирую остатки, потому что выше они в таблице в виде матрицы (242,5)

In [ ]:
shocks_df = pd.DataFrame(shocks, columns = ['brent', 'reer', 'inflation', 'miacr_1d', 'imoex'])
shocks_df
#shocks_df.head(2)

,brent,reer,inflation,miacr_1d,imoex
0,-2.113741,-2.176136,-0.258691,-0.053116,-1.290573
1,-1.776563,-4.427945,1.024822,-0.700060,1.842309
2,0.619178,3.080764,0.551314,-0.027084,1.356834
3,-0.186900,4.119863,0.253266,0.234480,0.638478
4,-0.346727,22.265785,-0.289958,1.925782,-1.140521
...,...,...,...,...,...
237,-0.346933,1.514327,-0.151347,0.031431,0.676040
238,-0.429928,-3.180948,-0.269598,-0.040524,-0.014412
239,-0.366991,5.564615,0.211053,0.292889,0.311171
240,0.502030,11.128168,1.291641,0.299416,1.133033


# LP + IRF когда все шоки в одной регрессии

$log(price_{t+h}) - log(price_{t-1}) = const + shocks$

In [ ]:
lp_table = pd.DataFrame(columns = ['ticker', 'shock', 'horizont', 'estimate', 'conf.low', 'conf.high'])
#lp_table = pd.concat([lp_table, pd.DataFrame({'horizont': 0, 'estimate': 1, 'conf.low': 1, 'conf.high': 1}, index=[0])])

for ticker in tickers_name:
  for h in range(0, 21):
      df_new = pd.DataFrame
      df_new = np.log(df_ticker[ticker])
      df_new['lag1'] = np.log(df_ticker[ticker].shift(h))


      X = pd.concat([df_new['lag1'], shocks_df.iloc[:,:4]], axis = 1)
      X = X[['brent', 'inflation', 'miacr_1d', 'reer']]
      X = sma.add_constant(X)
      y = df_ticker[ticker] - df_new['lag1']

      model = sma.OLS(y, X, missing = 'drop')
      results = model.fit()
      for sh in shocks_df.columns[:4]:
        lp_table = pd.concat([lp_table, pd.DataFrame({'ticker' : [ticker], 'shock': [sh], 'horizont': [h], 'estimate': results.params[sh],
                                                      'conf.low': results.conf_int()[0][sh], 'conf.high': results.conf_int()[1][sh]})])

In [ ]:
lp_table

,ticker,shock,horizont,estimate,conf.low,conf.high
0,SBER,brent,0,-300.853632,-2990.446358,2388.739093
0,SBER,reer,0,18.530775,-2564.075773,2601.137323
0,SBER,inflation,0,1616.822732,-9777.887697,13011.533162
0,SBER,miacr_1d,0,677.845627,-32192.561541,33548.252794
0,SBER,brent,1,-315.569493,-3043.450096,2412.311109
...,...,...,...,...,...,...
0,MSNG,miacr_1d,19,0.516394,-1.903904,2.936691
0,MSNG,brent,20,0.159314,-0.046383,0.365012
0,MSNG,reer,20,-0.052061,-0.243363,0.139242
0,MSNG,inflation,20,0.323944,-0.518411,1.166298


In [ ]:
filtered_df = lp_table[(lp_table['conf.high'] < 0)]
filtered_df2 = lp_table[(lp_table['conf.low'] > 0)]

# вывод уникальных комбинаций двух столбцов
unique_combinations = filtered_df[['ticker', 'shock']].drop_duplicates()
unique_combinations2 = filtered_df2[['ticker', 'shock']].drop_duplicates()
#unique_combinations

In [ ]:
pivot_table = pd.pivot_table(lp_table,
               index=["shock", "ticker"],
               values = "estimate",
               aggfunc=np.mean,
               sort = True)

pivot_table

estimate
shock ticker             
brent AFKS      -7.057251
      AFLT       3.275306
      AGRO       3.649334
      ALRS       1.923031
      CBOM      -0.022922
...                   ...
reer  TRNFP  -7368.934925
      UPRO      -0.014071
      UWGN     -70.606513
      VTBR       0.025675
      YNDX      77.022810

[192 rows x 1 columns]

In [ ]:
# Можно загрузить в эксельку и посмотреть там
#lp_table.to_excel('Шоки_reer.xlsx')

In [ ]:
# отельно таблицы для разных шоков
lp_table_brent = lp_table[lp_table['shock'] == 'brent']
lp_table_usd = lp_table[lp_table['shock'] == 'usd']
lp_table_inflation = lp_table[lp_table['shock'] == 'inflation']
lp_table_miacr = lp_table[lp_table['shock'] == 'miacr']

$price_{t+h} ~ price_{t-1} + const + shocks$

In [ ]:
# Тут я считаю IRF по LP (OLS)

#lp_table = pd.DataFrame(columns = ['ticker', 'shock', 'horizont', 'estimate', 'conf.low', 'conf.high'])
#lp_table = pd.concat([lp_table, pd.DataFrame({'horizont': 0, 'estimate': 1, 'conf.low': 1, 'conf.high': 1}, index=[0])])

#for ticker in tickers_name:
  #for h in range(1, 21):
      # df_new = pd.DataFrame
      # df_new = df_ticker[ticker]
      # df_new['lag1'] = df_ticker[ticker].shift(h)

      # X = pd.concat([df_new['lag1'], shocks_df.iloc[:,:4]], axis = 1)
      # X = sma.add_constant(X)

      # model = sma.OLS(df_ticker[ticker], X, missing = 'drop')
      # results = model.fit()
      # for sh in shocks_df.columns[:4]:
      #   lp_table = pd.concat([lp_table, pd.DataFrame({'ticker' : [ticker], 'shock': [sh], 'horizont': [h], 'estimate': results.params[sh],
      #                                                 'conf.low': results.conf_int()[0][sh], 'conf.high': results.conf_int()[1][sh]})])



## VECM

In [ ]:
from statsmodels.tsa.api import VECM

In [ ]:
model=SVECM(data, k_ar_diff=1, deterministic='na')
res=model.fit()
print(res.summary())

Det. terms outside the coint. relation & lagged endog. parameters for equation brent
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
L1.brent        -0.4076      0.064     -6.407      0.000      -0.532      -0.283
L1.reer          0.2952      0.177      1.672      0.095      -0.051       0.641
L1.inflation     0.0476      0.016      2.948      0.003       0.016       0.079
L1.miacr_1d      0.0013      0.004      0.289      0.773      -0.007       0.010
L1.imoex         0.2220      0.083      2.664      0.008       0.059       0.385
Det. terms outside the coint. relation & lagged endog. parameters for equation reer
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
L1.brent        -0.0461      0.021     -2.236      0.025      -0.086      -0.006
L1.reer          0.00

## ///

**Основные мысли по ходу кода**

1. Сбор данных
  - не все акции, но другие скорее всего короткие ряды за пару лет
  - если добавлять другие акции, то нужно искать в коде страницы id

2. VAR
  - шок посчитала $\epsilon_t (шок) = B^{-1}*U_t (остатки в var)$



**Основные выводы по результатам IRF**

Для около 30 компаний хотя бы 1 шок значим

Всего значимо примерно 47 шоков для разных акций

